# Mongo Data Validation - Restructuring the Proect

In [ ]:
// models/dish.js
//----------------------------------------------------------------
const Joi = require('joi');
const mongoose = require('mongoose');

const Dish = mongoose.model('Dish', new mongoose.Schema({
    name: {
        type: String,
        minlength: 3,
        required: true
    },
    price: {
        type: Number,
        required: true
    }
}));

function validateDish(dish){
    const schema = Joi.object({
        name: Joi.string()
            .min(3)
            .required(),
        price: Joi.number()
            .required()        
    })

    return schema.validate(dish);
}

module.exports.Dish = Dish;
module.exports.validate = validateDish;


//models/customer.js
//---------------------------------------------------------------------
const Joi = require('joi');
const mongoose = require('mongoose');

const Customer = mongoose.model('Customer', new mongoose.Schema({
    name: {
        type: String,
        minlength: 3,
        required: true
    },
    isGold: {
        type: Boolean,
        default: false,
    },
    phone: {
        type: String,
        required: true,
        minlength: 7
    }
}));

function validateCustomer(customer){
    const schema = Joi.object({
        name: Joi.string()
            .min(3)
            .required(),
        isGold: Joi.boolean(),
        phone: Joi.string()
            .min(7)
            .required()        
    })

    return schema.validate(customer);
}

module.exports.Customer = Customer;
module.exports.validate = validateCustomer;

In [ ]:
// router/dishes.js
//--------------------------------------------------------------
const {Dish, validate} = require('../models/dish')
const express = require('express');
const router = express.Router();


router.get('/', async (req, res) => {
    const dishes = await Dish.find().sort('name');
    res.send(dishes);
});

router.get('/:id', async (req, res) => {
    const dish = await Dish.findById(req.params.id);
    if (!dish) return res.status(404).send('The dish with the given ID was not found');
    
    res.send(dish);
});

router.post('/', async (req, res) => {
    const {error} = validate(req.body);
    if (error) return res.status(404).send(error.details[0].message);
    
    let dish = new Dish(req.body);
    dish = await dish.save();
    res.send(dish);
});

router.put('/:id', async (req, res) => {
    const {error} = validate(req.body);
    if (error) return res.status(404).send(error.details[0].message);

    const dish = await Dish.findByIdAndUpdate(req.params.id, req.body, {new: true}); 
    if (!dish) return res.status(404).send('The dish with the given ID was not found');
    
    res.send(dish);
});

router.delete('/:id', async (req, res) => {
    const dish = await Dish.findByIdAndDelete(req.params.id); 
    if (!dish) return res.status(404).send('The dish with the given ID was not found');
    
    res.send(dish);
});

module.exports = router;


//router/customers.js
//--------------------------------------------
const { Customer, validate } = require('../models/customer');
const express = require('express');
const router = express.Router();


router.get('/', async (req, res) => {
    const customers = await Customer.find().sort('name');
    res.send(customers);
});

router.get('/:id', async (req, res) => {
    const customer = await Customer.findById(req.params.id);
    if (!customer) return res.status(404).send('The customer with the given ID was not found');
    
    res.send(customer);
});

router.post('/', async (req, res) => {
    const {error} = validate(req.body);
    if (error) return res.status(404).send(error.details[0].message);
    
    let customer = new Customer(req.body);
    customer = await customer.save();
    res.send(customer);
});

router.put('/:id', async (req, res) => {
    const {error} = validate(req.body);
    if (error) return res.status(404).send(error.details[0].message);

    const customer = await Customer.findByIdAndUpdate(req.params.id, req.body, {new: true}); 
    if (!customer) return res.status(404).send('The customer with the given ID was not found');
    
    res.send(customer);
});

router.delete('/:id', async (req, res) => {
    const customer = await Customer.findByIdAndDelete(req.params.id); 
    if (!customer) return res.status(404).send('The customer with the given ID was not found');
    
    res.send(customer);
});

module.exports = router;
